<a href="https://colab.research.google.com/github/KGYAMFI22/csv-leaflet-map/blob/main/Comparison_of_Machine_Learning_Classifiers_in_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing Machine Learning Classifiers in GEE: An Introductory Guide with Python
## Introduction
The script aims to compare minimum distance, k-nearest neighbor (KNN), support vector machines, decision trees, random forest,and gradient tree boosting classifiers in Google Earth Engine (GEE). We use Python in Colab.

- Requirements
To run this script, the user must have an Earth Engine account. In addition, the user must authenticate the Earth Engine Python API. See the instructions [here](https://developers.google.com/earth-engine/guides/auth).
- This script will use the [geemap](https://geemap.org) Python package to visualize, and map land cover.

Following are the steps to compare the machine learning classifiers.

# Initialize and Authenticate Earth Engine
To get started with Google Earth Engine (GEE), you need to initialize and authenticate the Earth Engine API. Follow these steps.


First, import the Earth Engine API by importing the ee module into your Python environment. This module allows you to interact with the Earth Engine platform.

In [ ]:
# Import the EE API
import ee

Next, initialize the Earth Engine API. You must initialize the API to use Earth Engine functionalities. This involves authenticating your session and initializing the library. When you run the ee.Initialize() command for the first time, you might be prompted to authenticate your session. This will open a web browser window where you need to log in with your Google account and grant Earth Engine access.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-sahacstu') # Change to your EE project

## Import Libraries
Next, import the essential libraries needed to process and analyze the datasets.

In [ ]:
# Import the necessary libraries
import geemap

## Import the project boundary
First, import the study area boundary boundary.

In [ ]:
# Load the boundary
boundary = ee.Geometry.Polygon([
    [
        [100.08846791757806, 15.690328128622575],
        [100.14202626718743, 15.690328128622575],
        [100.14202626718743, 15.732301072397595],
        [100.08846791757806, 15.732301072397595],
        [100.08846791757806, 15.690328128622575]
    ]
])

# Load the boundary
training_areas = ee.FeatureCollection('projects/ee-sahacstu/assets/WorldCoverSample2020_Nakhonsawan')
print('Training Areas:', training_areas.size().getInfo())

Training Areas: 1400


## Prepare Sentinel-2 imagery
First, preprocess Sentinel-2 imagery by first applying cloud masking to remove cloud and cirrus pixels using the QA60 band. Then load Sentinel-2 surface reflectance images within a specified date range and filters them based on cloud cover, retaining images with less than 20% cloudiness. A median composite is generated from the filtered images and clipped to a specified boundary region. Finally, the composite is visualized as a false-color image using selected bands on an interactive map, complete with layer control for enhanced usabilit

In [ ]:
# Function for Cloud Masking
def mask_s2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load Sentinel-2
s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-03-01', '2022-06-30')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .filterBounds(boundary)  # <--- Restrict to your boundary
    .map(mask_s2clouds)
    .select(['B2','B3','B4','B5','B6','B7','B8','B11','B12'])
)

# Now let's see how many images we get
print("Number of images found:", s2.size().getInfo())

composite = s2.median().clip(boundary)


# Print Sentinel-2 Composite
print('Sentinel-2 Composite:', composite.getInfo())

# Display the Sentinel-2 composite
# Initialize the map
map1 = geemap.Map()

# Add the composite image to the map with specified display settings.
map1.addLayer(composite, {'bands': ['B11', 'B8', 'B3'], 'min': 0, 'max': 0.3}, 'Sentinel-2 Composite')

# Display the map with layer control.
map1.centerObject(boundary, 12)
map1.addLayerControl()
map1

Number of images found: 122
Sentinel-2 Composite: {'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [2, 2], 'origin': [99, 15], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [2, 2], 'origin': [99, 15], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [2, 2], 'origin': [99, 15], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [2, 2], 'origin': [99, 15], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.55350

Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…

## Prepare training data
In this step, we prepare the dataset for training and testing machine learning models by processing satellite imagery and training labels. We start by selecting Sentinel-2 bands (B2 to B12) and clipping the composite image to the specified boundary region, defining the input features. Next, we rasterize the vector training data using the Cl_Id property to create a raster layer representing class labels and add it as a new band (class) to the input features. To create a representative dataset, we use stratified sampling to extract reflectance values and class labels, ensuring proportional representation across classes. A random column is added to the dataset with a fixed seed for reproducibility, allowing us to split the data into 70% for training and 30% for validation. Finally, we confirm the dataset sizes to ensure the split is as intended. This process prepares the data for effective training and validation of machine learning models.

In [ ]:
# Load the boundary
training_areas = ee.FeatureCollection('projects/ee-sahacstu/assets/WorldCoverSample2020_Nakhonsawan')
print('Training Areas:', training_areas.size().getInfo())

training_areas = training_areas.select(
    ['class'],  # old property
    ['Cl_Id']   # new property
)

# Check how many features we have initially
num_features = training_areas.size().getInfo()
print("Number of features in training_areas:", num_features)

# ---------------------------------------------
# 5) Filter out features missing 'Cl_Id'
#    (In case some points don't have that property)
# ---------------------------------------------
valid_training_areas = training_areas.filter(ee.Filter.notNull(['Cl_Id']))

missing_clid_count = training_areas.size().subtract(valid_training_areas.size())
print("Features missing Cl_Id:", missing_clid_count.getInfo())

# We continue with valid_training_areas only
# so we don't get the "Property 'Cl_Id' is missing" error later
training_areas = valid_training_areas

# ---------------------------------------------
# 6) Use sampleRegions() to get pixel values
#    at each point
# ---------------------------------------------
bands = ['B2','B3','B4','B5','B6','B7','B8','B11','B12']

# Perform sampling
training_points = composite.sampleRegions(
    collection = training_areas,
    properties = ['Cl_Id'],  # Must match the property name in the points
    scale = 10               # Sentinel-2 resolution
)

count_sampled = training_points.size().getInfo()
print("Number of training points with imagery info:", count_sampled)

# If this is zero, check overlap or date range issues.

# ---------------------------------------------
# 7) Split data into training & testing
#    using a random column
# ---------------------------------------------
training_points = training_points.randomColumn('random', seed=50)
Sample_training = training_points.filter(ee.Filter.lt('random', 0.7))
Sample_test = training_points.filter(ee.Filter.gte('random', 0.7))

print("Number of training pixels:", Sample_training.size().getInfo())
print("Number of validation pixels:", Sample_test.size().getInfo())

# ---------------------------------------------
# 8) Train a Random Forest classifier
# ---------------------------------------------
rf_classifier = ee.Classifier.smileRandomForest(numberOfTrees=50).train(
    features=Sample_training,
    classProperty='Cl_Id',
    inputProperties=bands
)

# ---------------------------------------------
# 9) Classify the composite
# ---------------------------------------------
classified_map = composite.select(bands).classify(rf_classifier)

# ---------------------------------------------
# 10) Accuracy Assessment
# ---------------------------------------------
def compute_metrics(classifier, validation_data, label_prop):
    validated = validation_data.classify(classifier)
    error_matrix = validated.errorMatrix(label_prop, 'classification')
    oa = error_matrix.accuracy()
    ua = error_matrix.consumersAccuracy()
    pa = error_matrix.producersAccuracy()

    print('Error Matrix:\n', error_matrix.getInfo())
    print('Overall Accuracy:', oa.getInfo())
    print('User Accuracy:', ua.getInfo())
    print('Producer Accuracy:', pa.getInfo())

print("\n--- Accuracy Assessment (Random Forest) ---")
compute_metrics(rf_classifier, Sample_test, 'Cl_Id')

# ---------------------------------------------
# 11) Visualize in an interactive map
# ---------------------------------------------
Map = geemap.Map()
Map.centerObject(boundary, 12)

# Add Sentinel-2 composite for reference
Map.addLayer(
    composite,
    {'bands': ['B8','B11','B4'], 'min': 0, 'max': 0.3},
    'Sentinel-2 Composite'
)

# Add classification layer
palette = [
    "grey",   # class 0 example
    "red",    # class 1 example
    "yellow", # class 2 example
    "lime",   # class 3 example
    "green",  # class 4 example
    "blue",   # class 5 example
    "cyan",   # class 6 example
    "magenta",# class 7 example
    "orange", # class 8 example
    "brown"   # class 9 example
]
Map.addLayer(
    classified_map,
    {'min': 0, 'max': 9, 'palette': palette},
    'Random Forest Classification'
)

Map.addLayerControl()
Map

Training Areas: 1400
Number of features in training_areas: 1400
Features missing Cl_Id: 0
Number of training points with imagery info: 1400
Number of training pixels: 1001
Number of validation pixels: 399

--- Accuracy Assessment (Random Forest) ---
Error Matrix:
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
######################################################
# 1) Define the models we want to run
######################################################
models_to_run = [
    "Minimum distance",
    "KNN",
    "SVM",
    "CART",
    "Random forest",
    "Gradient tree boost"
]

######################################################
# 2) Create a function to instantiate the classifier
#    based on the string name
######################################################
def get_classifier(model_name):
    """
    Returns an ee.Classifier instance given a string name.
    """
    if model_name == "Minimum distance":
        return ee.Classifier.minimumDistance()
    elif model_name == "KNN":
        # K = 5 is a common choice, adjust as needed
        return ee.Classifier.smileKNN(k=5)
    elif model_name == "SVM":
        # RBF kernel, gamma=0.5 is just an example
        return ee.Classifier.libsvm(kernelType='RBF', gamma=0.5)
    elif model_name == "CART":
        # Basic CART decision tree
        return ee.Classifier.smileCart()
    elif model_name == "Random forest":
        # 100 trees is typical, can be tuned
        return ee.Classifier.smileRandomForest(numberOfTrees=100)
    elif model_name == "Gradient tree boost":
        # Example settings: 50 trees, small shrinkage
        return ee.Classifier.smileGradientTreeBoost(
            numberOfTrees=50,
            shrinkage=0.005,
            samplingRate=0.7,
            maxNodes=None,
            loss='LeastAbsoluteDeviation',
            seed=0
        )
    else:
        raise ValueError(f"Unknown model name: {model_name}")

######################################################
# 3) Function to train a classifier, classify the composite,
#    and compute accuracy metrics
######################################################
def run_classification(model_name, sample_training, sample_test, composite, bands):
    """
    1. Create the classifier using get_classifier().
    2. Train on Sample_training (which has Cl_Id).
    3. Classify the composite using the trained model.
    4. Compute accuracy metrics on Sample_test.
    5. Return the trained classifier, the classified image, and metrics.
    """
    # 1) Get the classifier
    classifier = get_classifier(model_name)

    # 2) Train the classifier
    trained_classifier = classifier.train(
        features=sample_training,
        classProperty='Cl_Id',
        inputProperties=bands
    )

    # 3) Classify the composite
    classified = composite.select(bands).classify(trained_classifier)

    # 4) Compute metrics on Sample_test
    validated = sample_test.classify(trained_classifier)
    error_matrix = validated.errorMatrix('Cl_Id', 'classification')

    overall_acc = error_matrix.accuracy().getInfo()
    user_acc = error_matrix.consumersAccuracy().getInfo()
    producer_acc = error_matrix.producersAccuracy().getInfo()

    metrics = {
        'error_matrix': error_matrix.getInfo(),
        'overall_accuracy': overall_acc,
        'user_accuracy': user_acc,
        'producer_accuracy': producer_acc
    }

    return trained_classifier, classified, metrics

######################################################
# 4) Loop over each model, store results, print metrics
######################################################
classification_results = {}
for model_name in models_to_run:
    trained_clf, classified_img, metrics = run_classification(
        model_name,
        Sample_training,      # your 70% training FC
        Sample_test,          # your 30% test FC
        composite,            # Sentinel-2 median composite
        bands                 # e.g. ['B2','B3','B4',...]
    )

    # Store the results in a dictionary
    classification_results[model_name] = {
        'classifier': trained_clf,
        'classified_image': classified_img,
        'metrics': metrics
    }

    # Print the accuracy metrics
    print(f"==== {model_name} ====")
    print("Error Matrix:\n", metrics['error_matrix'])
    print("Overall Accuracy:", metrics['overall_accuracy'])
    print("User Accuracy:",   metrics['user_accuracy'])
    print("Producer Accuracy:", metrics['producer_accuracy'])
    print("------------------------------------")

######################################################
# 5) (Optional) Visualize each classified result in geemap
#    or pick the best one.
######################################################
# For demonstration, let's display each classifier's output
# in a separate geemap Map. If you want to just show one,
# pick your favorite or highest accuracy.



==== Minimum distance ====
Error Matrix:
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
for model_name, result in classification_results.items():
    classified_map = result['classified_image']

    # Create a new map
    m = geemap.Map()
    m.centerObject(boundary, 12)

    # Add the composite for reference
    m.addLayer(
        composite,
        {'bands': ['B8','B11','B3'], 'min': 0, 'max': 0.3},
        'Sentinel-2 Composite'
    )

    # Add the classified image
    palette = ["grey","red","yellow","lime","green","blue","cyan","magenta","orange","brown"]
    m.addLayer(
        classified_map,
        {'min': 0, 'max': 9, 'palette': palette},
        f"{model_name} Classification"
    )

    m.addLayerControl()
    print(f"Displaying map for {model_name} classification...")
    display(m)

Displaying map for Minimum distance classification...


Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…

Displaying map for KNN classification...


Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…

Displaying map for SVM classification...


...

Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…

Displaying map for Random forest classification...


Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…

Displaying map for Gradient tree boost classification...


Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# ======================================
# HYPERPARAMETER TUNING FOR RANDOM FOREST
# ======================================

# We will try different combinations of numberOfTrees, variablesPerSplit, and minLeafPopulation.
# You can add/remove params or expand these ranges.
parameter_grid = [
    {'numberOfTrees': 50,  'variablesPerSplit': None, 'minLeafPopulation': 1},
    {'numberOfTrees': 50,  'variablesPerSplit': 2,    'minLeafPopulation': 1},
    {'numberOfTrees': 100, 'variablesPerSplit': None, 'minLeafPopulation': 1},
    {'numberOfTrees': 100, 'variablesPerSplit': 2,    'minLeafPopulation': 1},
    {'numberOfTrees': 100, 'variablesPerSplit': 2,    'minLeafPopulation': 2},
    {'numberOfTrees': 200, 'variablesPerSplit': 2,    'minLeafPopulation': 2},
]



best_accuracy = 0.0
best_params = None
best_classifier = None

# Loop over each set of hyperparameters
for params in parameter_grid:
    # Create a Random Forest with the current parameter set
    candidate_classifier = ee.Classifier.smileRandomForest(
        numberOfTrees      = params['numberOfTrees'],
        variablesPerSplit  = params['variablesPerSplit'],
        minLeafPopulation  = params['minLeafPopulation']
    ).train(
        features       = Sample_training,
        classProperty  = 'Cl_Id',
        inputProperties= bands
    )

    # Evaluate on the test/validation set
    validated = Sample_test.classify(candidate_classifier)
    error_matrix = validated.errorMatrix('Cl_Id', 'classification')
    overall_acc = error_matrix.accuracy().getInfo()

    print(f"Params: {params} | Accuracy: {overall_acc}")

    # Track the best model
    if overall_acc > best_accuracy:
        best_accuracy = overall_acc
        best_params = params
        best_classifier = candidate_classifier

print("\n=========== Hyperparameter Tuning Results ===========")
print("Best overall accuracy found:", best_accuracy)
print("Best parameters:", best_params)

# ======================================
# CLASSIFY WITH THE BEST CLASSIFIER
# ======================================
best_classified_map = composite.select(bands).classify(best_classifier)

# OPTIONAL: Print final confusion matrix using test data
best_validated = Sample_test.classify(best_classifier)
final_error_matrix = best_validated.errorMatrix('Cl_Id', 'classification')
print("\nFinal Error Matrix:\n", final_error_matrix.getInfo())
print("Final Overall Accuracy:", final_error_matrix.accuracy().getInfo())
print("User's Accuracy:", final_error_matrix.consumersAccuracy().getInfo())
print("Producer's Accuracy:", final_error_matrix.producersAccuracy().getInfo())

# ======================================
# VISUALIZE THE BEST CLASSIFICATION
# ======================================
best_map = geemap.Map()
best_map.centerObject(boundary, 12)

# Add original composite
best_map.addLayer(
    composite,
    {'bands': ['B8','B11','B4'], 'min': 0, 'max': 0.3},
    'Sentinel-2 Composite'
)

# Add the best classification
palette = ["grey","red","yellow","lime","green","blue","cyan","magenta","orange","brown"]
best_map.addLayer(
    best_classified_map,
    {'min': 0, 'max': 9, 'palette': palette},
    'Best RF Classification'
)

best_map.addLayerControl()
best_map


Params: {'numberOfTrees': 50, 'variablesPerSplit': None, 'minLeafPopulation': 1} | Accuracy: 0.5288220551378446
Params: {'numberOfTrees': 50, 'variablesPerSplit': 2, 'minLeafPopulation': 1} | Accuracy: 0.5388471177944862
Params: {'numberOfTrees': 100, 'variablesPerSplit': None, 'minLeafPopulation': 1} | Accuracy: 0.5263157894736842
Params: {'numberOfTrees': 100, 'variablesPerSplit': 2, 'minLeafPopulation': 1} | Accuracy: 0.543859649122807
Params: {'numberOfTrees': 100, 'variablesPerSplit': 2, 'minLeafPopulation': 2} | Accuracy: 0.5162907268170426
Params: {'numberOfTrees': 200, 'variablesPerSplit': 2, 'minLeafPopulation': 2} | Accuracy: 0.518796992481203

=========== Hyperparameter Tuning Results ===========
Best overall accuracy found: 0.543859649122807
Best parameters: {'numberOfTrees': 100, 'variablesPerSplit': 2, 'minLeafPopulation': 1}

Final Error Matrix:
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Map(center=[15.711315511054483, 100.11524709238084], controls=(WidgetControl(options=['position', 'transparent…